# 🧮 Урок 17: Первые проекты — Классификация MNIST с помощью scikit-learn
**Цель урока:** Научиться использовать готовые инструменты `scikit-learn` для решения реальных задач машинного обучения, таких как классификация рукописных цифр из датасета MNIST. Подходит для новичков.

## 📌 Что такое MNIST?
- **MNIST** — это набор данных, содержащий 70,000 изображений рукописных цифр от 0 до 9.
- Каждое изображение имеет размер 28x28 пикселей (всего 784 признака).
- Целевая переменная — цифра от 0 до 9.
- Используется как «Hello World» в компьютерном зрении и машинном обучении.

💡 **Почему MNIST?**
- Простой и понятный датасет для начинающих.
- Позволяет сосредоточиться на методах ML, а не на сложной обработке изображений.

In [ ]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt

# Загрузка MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

# Выводим информацию
print("Размер данных:", X.shape)  # (70000, 784)
print("Пример метки:", y[0])  # '5'

In [ ]:
# Преобразуем строку в изображение 28x28
some_digit = X[0].reshape(28, 28)
plt.imshow(some_digit, cmap="gray")
plt.axis("off")
plt.show()

## 🧱 Подготовка данных
### 1. Нормализация пикселей
- Пиксели изображения имеют значения от 0 до 255.
- Нормализуем их к диапазону [0, 1]:

In [ ]:
X = X / 255.0  # Деление на 255 приводит к диапазону [0, 1]

### 2. Разделение на обучающую и тестовую выборки
- Используем первые 60,000 образцов как обучающую выборку, оставшиеся 10,000 — тестовая.

In [ ]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

### 3. Случайное перемешивание данных
- Данные в `fetch_openml` уже перемешаны, но для других датасетов используйте:

In [ ]:
import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

## 🧠 Обзор моделей для классификации
### 1. Логистическая регрессия
- Как работает? Находит оптимальную гиперплоскость для разделения классов.
- Когда использовать? Для простых задач классификации, где данные линейно разделимы.
- Преимущества: Быстрая, интерпретируемая.
- Ограничения: Не подходит для сложных нелинейных зависимостей.

### 2. SVM (Метод опорных векторов)
- Как работает? Максимизирует зазор между классами.
- Когда использовать? Для задач с четкой границей между классами.
- Преимущества: Высокая точность на небольших наборах.
- Ограничения: Медленный на больших данных, требует настройки параметров.

### 3. kNN (k-ближайших соседей)
- Как работает? Классифицирует объект на основе соседей.
- Когда использовать? Для задач с небольшими данными и нелинейными границами.
- Преимущества: Простая реализация, работает без явного обучения.
- Ограничения: Медленный на больших данных, чувствителен к масштабу признаков.

💡 **Совет:** Для MNIST лучше всего подходят SVM и Random Forest, но начнем с логистической регрессии для простоты.

## 📊 Метрики качества
- **Accuracy (точность):** Доля правильных предсказаний в классификации.
- **Precision (точность классификации):** Сколько правильных положительных предсказаний среди всех положительных.
- **Recall (полнота):** Сколько правильных положительных предсказаний среди всех реальных положительных.
- **F1-score:** Гармоническое среднее между precision и recall.
- **Confusion Matrix:** Матрица, показывающая, сколько объектов какого класса предсказано как.
- **ROC-AUC:** Для бинарной классификации, но не подходит для многоклассовой.

### Подробнее о метриках:
#### Accuracy (точность)
```python
accuracy = (TP + TN) / (TP + TN + FP + FN)
```
- **TP:** Истинно положительные (правильно предсказанные положительные).
- **TN:** Истинно отрицательные (правильно предсказанные отрицательные).
- **FP:** Ложноположительные (ошибка 1-го рода).
- **FN:** Ложноотрицательные (ошибка 2-го рода).

#### Precision (точность классификации)
```python
precision = TP / (TP + FP)
```
- **Высокий precision:** Мало ложных срабатываний (например, модель редко ошибается, но может пропустить реальные случаи).

#### Recall (полнота)
```python
recall = TP / (TP + FN)
```
- **Высокий recall:** Мало пропущенных реальных случаев (например, важно не пропустить болезнь).

#### F1-score
- **Формула:**
  ```python
  F1 = 2 * (precision * recall) / (precision + recall)
  ```
- **Использование:** Когда нужно сбалансировать precision и recall.

## 🧪 Обучение логистической регрессии
### Шаг 1: Обучение модели
```python
from sklearn.linear_model import LogisticRegression

# Обучение
log_reg = LogisticRegression(max_iter=1000, solver='lbfgs')
log_reg.fit(X_train, y_train)
```

### Шаг 2: Оценка качества
```python
from sklearn.metrics import accuracy_score

# Предсказание
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')  # Пример: 0.92
```

### Шаг 3: Анализ ошибок
```python
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
```

### Шаг 4: Визуализация ошибочных предсказаний
```python
import numpy as np

# Найдем ошибки
wrong_indices = np.where(y_test != y_pred)[0][:5]
for idx in wrong_indices:
    plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f'Actual: {y_test[idx]}, Predicted: {y_pred[idx]}')
    plt.axis('off')
    plt.show()
```

## 📊 Практика: Сравнение моделей
### 1. Обучение SVM
```python
from sklearn.svm import SVC

# Обучение
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

# Оценка
svm_accuracy = accuracy_score(y_test, svm_clf.predict(X_test))
print(f'SVM Accuracy: {svm_accuracy:.2f}')
```

### 2. Обучение kNN
```python
from sklearn.neighbors import KNeighborsClassifier

# Обучение
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)

# Оценка
knn_accuracy = accuracy_score(y_test, knn_clf.predict(X_test))
print(f'kNN Accuracy: {knn_accuracy:.2f}')
```

### 3. Сравнение моделей
```python
import matplotlib.pyplot as plt

# График точности
models = ['Logistic Regression', 'SVM', 'kNN']
accuracies = [accuracy, svm_accuracy, knn_accuracy]

plt.bar(models, accuracies, color=['blue', 'green', 'orange'])
plt.ylabel('Accuracy')
plt.title('Model Comparison on MNIST')
plt.ylim(0.9, 1.0)
plt.show()
```

## 📈 Глубокий анализ модели
### 1. Precision, Recall, F1-score
```python
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)
```

### 2. ROC-AUC (только для бинарной классификации)
```python
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Бинаризация меток
y_test_binary = label_binarize(y_test, classes=np.unique(y_test))
y_pred_binary = log_reg.predict_proba(X_test)

# ROC-AUC
roc_auc = roc_auc_score(y_test_binary, y_pred_binary, multi_class='ovr')
print(f'ROC-AUC: {roc_auc:.2f}')
```

## 🧪 Дополнительная практика: настройка гиперпараметров
### 1. GridSearchCV для kNN
```python
from sklearn.model_selection import GridSearchCV

# Параметры для перебора
param_grid = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}

# Поиск лучшей модели
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy')
grid.fit(X_train[:1000], y_train[:1000])  # Используем часть данных для ускорения

# Лучшие параметры
print(grid.best_params_)
```

## 📝 Домашнее задание
**Задача 1:** Обучите модель `KNeighborsClassifier` на MNIST и сравните её с логистической регрессией и SVM.
**Задача 2:** Напишите короткий отчет (200–300 слов), где:
- Опишите, как вы обучали модель.
- Сравните точность разных моделей.
- Объясните, почему одни модели работают лучше других.
- Приведите примеры ошибочных предсказаний.

💡 **Рекомендации:**
- Используйте `KNeighborsClassifier(n_neighbors=3)` для начала.
- Для ускорения можно уменьшить размер выборки (например, `X_train[:1000]`).
- Для визуализации ошибок используйте `confusion_matrix` и `matplotlib`.

## 🧠 Дополнительная теория: Почему важна нормализация?
- **Нормализация** — это приведение данных к общему масштабу.
- **Зачем?** Многие алгоритмы (например, kNN, SVM) чувствительны к масштабу признаков.
- **Методы нормализации:**
  - **MinMaxScaler:** Приводит к диапазону [0, 1].
  - **StandardScaler:** Приводит к среднему 0 и дисперсии 1.
- **Пример:**
  ```python
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)
  ```

## 📉 Как работает SVM?
- **SVM (Support Vector Machine)** — это алгоритм, который ищет гиперплоскость, максимально разделяющую классы.
- **Hard Margin:** Идеальное разделение (но не подходит для реальных данных).
- **Soft Margin:** Позволяет некоторым объектам быть внутри границы.
- **Kernel Trick:** Позволяет работать с нелинейными границами (например, RBF).
- **Плюсы:** Высокая точность на небольших наборах.
- **Минусы:** Медленный на больших данных, чувствителен к параметрам.

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(X_train[:5000], y_train[:5000])  # Используем часть данных для ускорения
print(svm.score(X_test[:1000], y_test[:1000]))  # Точность на подвыборке

## 🧲 Как работает kNN?
- **kNN (k-ближайших соседей):**
  1. Сохраняет всю обучающую выборку.
  2. При предсказании находит `k` ближайших соседей.
  3. Присваивает класс, который чаще всего встречается среди соседей.
- **k=1:** Просто берет ближайший объект.
- **k=5:** Усредняет результаты 5 ближайших объектов.
- **Плюсы:** Простая реализация, работает без явного обучения.
- **Минусы:** Медленно работает на больших данных, чувствителен к масштабу признаков.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Обучение
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train[:5000], y_train[:5000])

# Оценка
accuracy = accuracy_score(y_test[:1000], knn.predict(X_test[:1000]))
print(f'Accuracy: {accuracy:.2f}')  # Пример: 0.95

## 📊 Визуализация матрицы ошибок
### 1. Confusion Matrix
- **Что это?** Таблица, показывающая, сколько объектов какого класса предсказано как.
- **Как читать?**
  - Диагональ — правильные предсказания.
  - Вне диагонали — ошибки.
- **Пример:**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## 📈 Как работает градиентный спуск в логистической регрессии
- **Градиентный спуск:** Алгоритм минимизации функции потерь.
- **Идея:** Двигайтесь в направлении убывания градиента.
- **Формула обновления весов:**
  ```python
  weights -= learning_rate * gradient
  ```
- **learning_rate:** Шаг обучения. Слишком большой — модель не сходится, слишком маленький — медленно обучается.
- **solver='lbfgs':** Используется оптимизация на основе алгоритма L-BFGS.
- **max_iter=1000:** Количество итераций для сходимости.

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000, solver='lbfgs')
log_reg.fit(X_train, y_train)
print(log_reg.score(X_test, y_test))  # Точность на тестовой выборке

## 📉 Как работает кросс-валидация?
- **Кросс-валидация (Cross-Validation):** Разделение данных на `k` частей для оценки модели.
- **Зачем?** Чтобы проверить, как модель будет работать на разных подвыборках.
- **Пример:**

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg, X_train, y_train, cv=3, scoring='accuracy')
print(scores)  # [0.91, 0.92, 0.93]
print(f'Средняя точность: {scores.mean():.2f}')

## 📊 Как выбрать лучшую модель?
- **Точность (Accuracy):** Подходит для сбалансированных классов.
- **F1-score:** Если важны и precision, и recall (например, редкие классы).
- **ROC-AUC:** Для бинарной классификации.
- **Скоринг в GridSearchCV:** Используйте параметр `scoring` для выбора метрики.

In [ ]:
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy')

## 🧪 Практика: Обучение и оценка
### Шаг 1: Обучение логистической регрессии
```python
from sklearn.linear_model import LogisticRegression
import numpy as np

# Обучение
log_reg = LogisticRegression(max_iter=1000, solver='lbfgs')
log_reg.fit(X_train, y_train)

# Предсказание
y_pred = log_reg.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
```

### Шаг 2: Обучение SVM
```python
from sklearn.svm import SVC

# Обучение
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train[:5000], y_train[:5000])  # Используем подвыборку для ускорения

# Оценка
svm_accuracy = accuracy_score(y_test[:1000], svm_clf.predict(X_test[:1000]))
print(f'SVM Accuracy: {svm_accuracy:.2f}')
```

### Шаг 3: Обучение kNN
```python
from sklearn.neighbors import KNeighborsClassifier

# Обучение
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train[:5000], y_train[:5000])

# Оценка
knn_accuracy = accuracy_score(y_test[:1000], knn_clf.predict(X_test[:1000]))
print(f'kNN Accuracy: {knn_accuracy:.2f}')
```

### Шаг 4: Сравнение моделей
```python
import matplotlib.pyplot as plt

# График точности
models = ['Logistic Regression', 'SVM', 'kNN']
accuracies = [accuracy_score(y_test, y_pred), svm_accuracy, knn_accuracy]

plt.bar(models, accuracies, color=['blue', 'green', 'orange'])
plt.ylabel('Accuracy')
plt.title('Model Comparison on MNIST')
plt.ylim(0.9, 1.0)
plt.show()
```

## 📝 Домашнее задание
**Задача 1:** Обучите модель `KNeighborsClassifier` на MNIST и сравните её с логистической регрессией и SVM.
**Задача 2:** Напишите короткий отчет (200–300 слов), где:
- Опишите, как вы обучали модель.
- Сравните точность разных моделей.
- Объясните, почему одни модели работают лучше других.
- Приведите примеры ошибочных предсказаний.
- Объясните, как можно улучшить модель (например, через нормализацию, подбор гиперпараметров).
- Нарисуйте графики точности для разных значений `n_neighbors`.
- Визуализируйте confusion matrix для `kNN`.

## ✅ Рекомендации по выполнению домашнего задания
- **Задача 1:** Убедитесь, что вы используете правильный диапазон `n_neighbors` (например, от 1 до 10).
- **Задача 2:** Попробуйте разные значения `n_neighbors` и посмотрите, как меняется точность.
- **Визуализация:** Используйте `sns.heatmap` для матрицы ошибок.
- **Подсказка:** Чем больше `n_neighbors`, тем меньше шума, но выше вычислительная сложность.